In [1]:
import pandas as pd
from scipy.stats import multivariate_normal
import seaborn as sns
import numpy as np

In [2]:
train_data = pd.read_csv('ML _dataset/BreastCancerTrain.csv')
valid_data = pd.read_csv('ML _dataset/BreastCancerValidation.csv')

In [3]:
train_data

,5,1,1.1,3,2,1.2,1.3,1.4,1.5,0
0,2,1,1,1,2,1,3,1,1,0
1,3,4,5,3,7,3,4,6,1,0
2,2,7,10,10,7,10,4,9,4,1
3,1,1,1,1,2,1,2,1,1,0
4,4,1,1,1,3,1,2,2,1,0
...,...,...,...,...,...,...,...,...,...,...
341,3,1,1,1,3,2,1,1,1,0
342,2,1,1,1,2,1,1,1,1,0
343,5,10,10,3,7,3,8,10,2,1
344,4,8,6,4,3,4,10,6,1,1


In [4]:
train_data0 = train_data[train_data['0'] == 0]  # classify by 0
X_0 = train_data0.drop('0',axis = 1)
train_data1 = train_data[train_data['0'] == 1]# classify by 1
X_1 = train_data1.drop('0',axis = 1)
X0 = X_0.to_numpy().tolist()
X1 = X_1.to_numpy().tolist()

In [5]:
valid_data0 = valid_data[valid_data['0'] == 0]
X_0valid = valid_data0.drop('0',axis = 1)
valid_data1 = valid_data[valid_data['0']==1]
X_1valid = valid_data1.drop('0',axis = 1)
X0valid = X_0valid.to_numpy().tolist()
X1valid = X_1valid.to_numpy().tolist()

In [6]:
prior_0 = len(X0) / (len(X1) + len(X0))
prior_1 = len(X1) / (len(X1) + len(X0))
prior_0

0.7630057803468208

In [7]:
mu0 = np.mean(X0, axis=0)
mu1 = np.mean(X1, axis=0)
cov0 = np.cov(np.transpose(X0))   #Use transpose(a, argsort(axes)) to invert the transposition of tensors when using the axes keyword argument.
cov1 = np.cov(np.transpose(X1))

In [8]:
import scipy.stats
def class_prob(label, x):

    if label == '1':
    # Priors
        class_prior, other_prior = prior_0, prior_1
        mu, other_mu = mu1, mu0
        cov, other_cov = cov1, cov0
    else:
    # Priors
        class_prior,other_prior = prior_1, prior_0
        mu, other_mu = mu0, mu1
        cov, other_cov = mu0, mu1
    
  # Class conditional densitites (or likelihoods)
    
    p_x_given_class = scipy.stats.multivariate_normal.pdf(x, mu, cov)
    p_x_given_other_class = scipy.stats.multivariate_normal.pdf(x, other_mu, other_cov)

  # Evidence
    evidence = p_x_given_class * class_prior + p_x_given_other_class * other_prior

    return p_x_given_class * class_prior / evidence


In [9]:
class_prob('0',X0[6])

0.9999999993466007

In [10]:
def test_accuracy(one, zero):
    false_zero = false_one = true_one = true_zero = 0

    for sample in zero:
        prob_one = class_prob('1', sample)
        if prob_one >= 0.5:
            true_one += 1
        else:
            false_zero += 1
  
    for sample in one:
        prob_zero = class_prob('0', sample)
        if prob_zero >= 0.5:
            true_zero += 1
        else:
            false_one += 1
  
    accuracy = (true_one + true_zero) / (true_one + true_zero + false_one + false_zero)
  
    return accuracy, true_one, true_zero, false_one, false_zero

In [11]:
test_accuracy(X0, X1)

(0.9826589595375722, 82, 258, 6, 0)

In [12]:
test_accuracy(X0valid,X1valid)

(0.9701492537313433, 157, 168, 10, 0)